In [1]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = '../data'
CITIBIKE_CSV_DIR = f'{DATA_DIR}/citibike/csv'
EXPORTS_DIR = f'{DATA_DIR}/citibike/exports'

In [3]:
DATE_PREFIXES = np.load(f'{DATA_DIR}/citibike/date_prefixes.npy')

In [4]:
stations_df = pd.read_csv(f'{EXPORTS_DIR}/station_coords.csv')
station_ids = stations_df['id'].unique()
len(station_ids)

3167

In [5]:
def is_integer_num(n):
    if isinstance(n, int):
        return True
    if isinstance(n, float):
        return n.is_integer()
    return False

def hash_station_id(station_id):
    if is_integer_num(station_id):
        station_id = int(station_id)
    return str(station_id)

In [7]:
for date_prefix in tqdm(DATE_PREFIXES):
    df = pd.read_csv(f'{CITIBIKE_CSV_DIR}/{date_prefix}-citibike-tripdata.csv')
    df = df.rename(
        columns={
            'start station id': 'start_station_id',
            'start station latitude': 'start_lat',
            'start station longitude': 'start_lon',
            'end station id': 'end_station_id',
            'end station latitude': 'end_lat',
            'end station longitude': 'end_lon',
            'Start Station ID': 'start_station_id',
            'Start Station Latitude': 'start_lat',
            'Start Station Longitude': 'start_lon',
            'End Station ID': 'end_station_id',
            'End Station Latitude': 'end_lat',
            'End Station Longitude': 'end_lon',
            'start_lng': 'start_lon',
            'end_lng': 'end_lon',
        })

    # Standarize station ID.
    df['start_station_id'] = df['start_station_id'].apply(
        lambda row: hash_station_id(row))
    df['end_station_id'] = df['end_station_id'].apply(
        lambda row: hash_station_id(row))
    df = df.sort_values(by=['start_station_id', 'end_station_id'])

    # Group trips by origin and destination. Aggregate trip count.
    count_df = df[['start_station_id', 'end_station_id']].reset_index()
    count_df = count_df.groupby(
        by=['start_station_id', 'end_station_id']).count().reset_index()
    count_df = count_df.rename(columns={'index': 'count'})

    # Convert to adjacency matrix. Origins are rows, destinations are columns.
    adj_matrix = pd.crosstab(count_df['start_station_id'],
                             count_df['end_station_id'],
                             count_df['count'],
                             aggfunc=np.sum,
                             dropna=False)
    adj_matrix = adj_matrix.fillna(0)
    adj_matrix[adj_matrix.columns] = adj_matrix[adj_matrix.columns].astype(int)

    # Remove extra index information.
    ids = adj_matrix.columns.union(adj_matrix.index).sort_values()
    adj_matrix = adj_matrix.reindex(index=ids, columns=ids, fill_value=0)

    adj_matrix.to_csv(f'{EXPORTS_DIR}/adjacency_matrices/' +
                      f'{date_prefix}_trip_count.csv')
    print(f'Wrote {EXPORTS_DIR}/adjacency_matrices/' +
          f'{date_prefix}_trip_count.csv; ' + f'len: {len(adj_matrix)}')

  1%|          | 1/97 [00:04<06:39,  4.16s/it]

Wrote ../data/citibike/exports/adjacency_matrices/201310_trip_count.csv; len: 330


  2%|▏         | 2/97 [00:06<05:13,  3.30s/it]

Wrote ../data/citibike/exports/adjacency_matrices/201311_trip_count.csv; len: 330


  3%|▎         | 3/97 [00:08<04:01,  2.57s/it]

...

Wrote ../data/citibike/exports/adjacency_matrices/202108_trip_count.csv; len: 1591


 99%|█████████▉| 96/97 [09:55<00:16, 16.86s/it]

Wrote ../data/citibike/exports/adjacency_matrices/202109_trip_count.csv; len: 1597


100%|██████████| 97/97 [10:12<00:00,  6.32s/it]

Wrote ../data/citibike/exports/adjacency_matrices/202110_trip_count.csv; len: 1608
